**notebook um mit attentionmaps von dino und svm mit lineraem und rvf kernel masken zu berechnen und evaluierung dieser**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [ ]:
import os
os.chdir('/content/BA')

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [ ]:
arch = 'vit_base'
patch_size = 8

output_dir = 'OUTPUT_DIR'
    
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
import numpy as np

def crop_fkt(bbox, area, percentile, width, height):
  width = width
  height = height

  width_edge = bbox[2]*percentile
  height_edge = bbox[3]*percentile

  edge = (width_edge + height_edge)

  tx = max(bbox[0]-edge, 0)
  ty = max(bbox[1]-edge, 0)
  bx = min(bbox[0]+bbox[2]+edge, width)
  by = min(bbox[1]+bbox[3]+edge, height)

  area = 80000
  wid = bx - tx
  hei = by - ty

  p = np.sqrt(area/(wid*hei))

  imagewidth = int((int(wid*p))/10)
  imageheight = int((int(hei*p))/10)

  img_size = (imageheight, imagewidth)

  return (tx, ty, bx, by), img_size

In [ ]:
def gt_tensors(attentions, maskimg, bbox, width, height):
  percentile = 0 #percentile hier 0, da attentionmaps schon zugeschnitten sind
  patch_size = 8
  image_size = (480, 480)
  area = 60000

  box, img_size = crop_fkt(bbox, area, percentile, width, height)

  img_size = (16, 16)

  crop_mask = maskimg.crop(box=box)

  mask = np.asarray(crop_mask)
  mask = torch.tensor(mask)
  mask = mask.reshape(1, mask.shape[0], mask.shape[1]).float()

  masktransforms = pth_transforms.Compose([
                                      pth_transforms.Resize(img_size, interpolation = InterpolationMode('nearest')),
                                     ])
  
  atttransforms = pth_transforms.Compose([
                                      pth_transforms.Resize(img_size),
                                     ])

  attentions = atttransforms(attentions)
  mask = masktransforms(mask)

  return attentions, mask

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import InterpolationMode

class NACTI_seg(Dataset):
  def __init__(self, annotations_file, dataset_dir):
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file) 

    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()
    self.ann_ids = self.coco.getAnnIds()


  def __len__(self):
    return len(self.ann_ids)

  def __getitem__(self, idx):

    ann_id = self.ann_ids[idx]
    ann = self.coco.loadAnns([ann_id])
    img_id = ann[0]['image_id']
    img = self.coco.loadImgs([img_id])

    img_path = self.dataset + '/images/' + img[0]['file_name']
    image = None
    mask = None
    bbox = None#

    width = img[0]['width']
    height = img[0]['height']

    attentions_path = self.dataset + '/tensors/' + str(ann_id) + '_attention.pt'
    attentions = torch.load(attentions_path, map_location = device)
    
    mask = self.coco.annToMask(ann[0]) #mask for instance segmentation
    bbox = ann[0]['bbox']

    maskImg = Image.fromarray(mask)

    attentions, mask = gt_tensors(attentions, maskImg, bbox, width, height)

    attentions = attentions.to(device)
    mask = mask.to(device)

    return attentions, mask, ann[0]

In [ ]:
trainset = NACTI_seg('TRAINSET__ANNOTATION_JSON', 'DATASET_PATH')

In [ ]:
train_ids = np.arange(0,trainset.__len__(),1)

X = []
Y = []

for id in train_ids:
  x, y, ann = trainset.__getitem__(id)

  mask_shape = y.shape

  Xarray = np.asarray(x)
  Yarray = np.asarray(y.reshape(mask_shape[1], mask_shape[2]))

  X.append(Xarray.transpose(1,2,0))
  Y.append(Yarray)  

  output_dir = '/content'

  os.makedirs(output_dir, exist_ok=True)                              # |||| maybe try no normalize

In [ ]:
X_arr = np.asarray(X)
Y_arr = np.asarray(Y)

In [ ]:
from sklearn import svm

n_attention_maps = X_arr.shape[3]

clf = svm.SVC(cache_size = 20000, max_iter = 1000000) #trainingsparameter für rbf svm
#clf = svm.LinearSVC()  #training für lin svm
X_svm = X_arr.reshape(-1, n_attention_maps)
Y_svm = Y_arr.reshape(-1) * 1
clf.fit(X_svm, Y_svm)

In [ ]:
from joblib import dump, load

dump(clf, 'FILE.joblib') #speichern

In [ ]:
drive.flush_and_unmount()

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# testing

In [ ]:
from __future__ import division
from __future__ import print_function


import os
import cv2
import json, yaml
import numpy as np
from PIL import Image
from collections import OrderedDict
from pycocotools import mask as cocomask
from pycocotools import coco as cocoapi

#binäre masken umwandeln um in json speichern zu können
def __get_annotation__(mask, image=None):

  labelMask = np.expand_dims(mask, axis=2)
  labelMask = labelMask.astype('uint8')
  labelMask = np.asfortranarray(labelMask)
  Rs = cocomask.encode(labelMask)
  assert len(Rs) == 1
  Rs = Rs[0]

  area = cocomask.area(Rs)

  return Rs, area#, [x, y, w, h] 

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import InterpolationMode

class TestSet(Dataset):
  def __init__(self, annotations_file, dataset_dir):
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file)  #for fast reading
    
    #self.transform = transform
    #self.target_transform = target_transforms

    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()
    self.ann_ids = self.coco.getAnnIds()


  def __len__(self):
    return len(self.ann_ids)

  def __getitem__(self, idx):

    ann_id = self.ann_ids[idx]
    ann = self.coco.loadAnns([ann_id])
    img_id = ann[0]['image_id']
    img = self.coco.loadImgs([img_id])

    img_path = self.dataset + '/images/' + img[0]['file_name']
    image = None
    mask = None
    bbox = None#
    
    width = img[0]['width']
    height = img[0]['height']

    attentions_path = self.dataset + '/tensors/' + str(ann_id) + '_attention.pt'
    attentions = torch.load(attentions_path, map_location = device)
    
    mask = self.coco.annToMask(ann[0])
    #mask = np.zeros((3000,3000))
    bbox = ann[0]['bbox']

    maskImg = Image.fromarray(mask)

    attentions = attentions.to(device)

    return attentions, mask, ann[0], bbox, (height, width)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

testset = TestSet('ANNOTATION_JSON', 'DATASET_PATH')

In [ ]:
from joblib import dump, load
clf = load('MODEL_FILE_joblib') 

In [ ]:
#inferenz mit rbfSVM

test_ids = np.arange(0,testset.__len__(),1)

Ygt = []
Ypred = []

Ygt = []
Ypred = []

#smal_pred = []

pred_anns = []

for i in test_ids:

  attentions, gtmask, annotation, bbox, dimensions = testset.__getitem__(i)

  Ygt.append(gtmask)
  
  area = 6400
  wid = bbox[3]
  hei = bbox[2]

  p = np.sqrt(area/(wid*hei))
  pre_dim = (int(bbox[3]*p), int(bbox[2]*p))


  transforms = pth_transforms.Compose([ 
                                        pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                        pth_transforms.Resize(pre_dim),
                                        pth_transforms.GaussianBlur(kernel_size=11, sigma=4),#pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                      ])
  attentions = transforms(attentions)

  attentions = np.asarray(attentions).transpose(1,2,0)
  org_size = (attentions.shape[0], attentions.shape[1])

  attentions = attentions.reshape(-1, 12)

  mask_part = clf.predict(attentions)
  mask_part = mask_part.reshape(org_size)

  mask = Image.fromarray(mask_part)
  
  transforms = pth_transforms.Compose([ 
                                        pth_transforms.Resize((int(bbox[3]), int(bbox[2])), interpolation = InterpolationMode('nearest')),
                                      ])
  mask = transforms(mask)

  mask_part = np.asarray(mask)
  pred_mask = np.zeros((dimensions))


  for i in range(mask_part.shape[0]):
    for j in range(mask_part.shape[1]):
      pred_mask[int(bbox[1]) + i, int(bbox[0]) + j] = mask_part[i, j]


  seg, area = __get_annotation__(pred_mask)

  annotation['pred_seg'] = seg
  annotation['pred_seg']['counts'] = annotation['pred_seg']['counts'].decode()
  annotation['pred_area'] = int(area)

  pred_anns.append(annotation)

  Ypred.append(pred_mask)


In [ ]:
#inferenz mit linSVM

test_ids = np.arange(0,testset.__len__(),1)

Ygt = []
Ypred = []


pred_anns = []

for i in test_ids:
  attentions, gtmask, annotation, bbox, dimensions = testset.__getitem__(i)

  Ygt.append(gtmask)

  transforms = pth_transforms.Compose([ 
                                        pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                        pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                        pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                        pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                        pth_transforms.Resize((int(bbox[3]), int(bbox[2]))),
                                        pth_transforms.GaussianBlur(kernel_size=11, sigma=4),#pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                        pth_transforms.GaussianBlur(kernel_size=11, sigma=4),
                                        pth_transforms.GaussianBlur(kernel_size=11, sigma=4),#pth_transforms.GaussianBlur(kernel_size=3, sigma=0.5),
                                      ])
  attentions = transforms(attentions)
  attentions = np.asarray(attentions).transpose(1,2,0)
  org_size = (attentions.shape[0], attentions.shape[1])
  attentions = attentions.reshape(-1, 12)

  mask_part = clf.predict(attentions)

  mask_part = mask_part.reshape(org_size)
  pred_mask = np.zeros(dimensions, dtype=np.int8)

  for i in range(mask_part.shape[0]):
    for j in range(mask_part.shape[1]):
      pred_mask[int(bbox[1]) + i, int(bbox[0]) + j] = mask_part[i, j]

  seg, area = __get_annotation__(pred_mask)

  annotation['pred_seg'] = seg
  annotation['pred_seg']['counts'] = annotation['pred_seg']['counts'].decode()
  annotation['pred_area'] = int(area)

  pred_anns.append(annotation)
  Ypred.append(pred_mask)

In [ ]:
drive.flush_and_unmount()

In [ ]:
from sklearn.metrics import jaccard_score as IoU
from sklearn.metrics import accuracy_score

iouscores = []

accscores = []

tempgt = Ygt #
temp = Ypred #

for i in range(len(tempgt)):
  gt = np.copy(tempgt[i])
  pred = np.copy(temp[i])

  iou = IoU(gt, pred, average='micro')

  acc = accuracy_score(gt, pred)

  accscores.append(acc)
  iouscores.append(iou)

In [ ]:
summ = 0

for i in range(len(iouscores)):
  summ = summ + iouscores[i]
print(summ)
summ = summ/len(iouscores)

print(summ)

35.38767501704155
0.5529324221412742


In [ ]:
summ = 0

for i in range(len(accscores)):
  summ = summ + accscores[i]
print(summ)
summ = summ/len(accscores)

print(summ)

184.9843605324073
0.722595158329716


In [ ]:
import json 

output_file = 'RESULTS_JSON'

with open(output_file, 'w') as file:
  json.dump(pred_anns, file, indent = 4, sort_keys=False)